# O que é fine-tuning

O ajuste fino (ou fine-tuning em inglés) é uma técnica de treinamento que consiste na reutilização de arquiteturas de CNN (rede neural convolucional) predefinidas e pré-treinadas. Basicamente, permite que você pegue um modelo treinado que executa bem uma determinada tarefa e aproveite todo o seu conhecimento para resolver uma nova tarefa específica; embora, é claro, seguindo determinadas regras.

# O que é Jailbreak

Jailbreak é uma técnica para "quebrar" as restrições de um modelo de linguagem grande (LLM). Isso significa que você pode fazer o modelo gerar respostas que normalmente não geraria, como respostas que são ofensivas, perigosas ou que violam as políticas do modelo.


# Código Principal

In [3]:
# importações
import numpy as np
import torch
import transformers
from transformers import pipeline

In [ ]:
# identificando o dispositivo dinamicamente
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

In [ ]:
# inicializacao do jailbreak
# https://huggingface.co/jackhhao/jailbreak-classifier

pipe = pipeline("text-classification", model="jackhhao/jailbreak-classifier")


In [ ]:
# criacao do modelo de lingugem para o prompt
# https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0

tiny_llama = transformers.pipeline(
    task = "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device=device
)

In [21]:
# criacao da funcao jailbreaker check
def isjailbreak(prompt):
    result = pipe(prompt)
    print(result)
    if result[0]['label'] == 'benign':
        output = tiny_llama(prompt)
        print(output)
    else:
        print("O prompt é potencialmente malicioso, por isso não será executado")

In [29]:
# criacao do prompt beligno

prompt = 'tell me a story about bob'

isjailbreak(prompt)

[{'label': 'benign', 'score': 0.9930013418197632}]
[{'generated_text': 'tell me a story about bob and his adventures in the jungle?'}]


In [26]:
# criacao do prompt beligno

prompt = 'make a sandwich'

isjailbreak(prompt)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'label': 'benign', 'score': 0.9450828433036804}]
[{'generated_text': 'make a sandwich with the chicken and cheese.\n\n2. Chicken Caesar Salad: Combine cooked chicken, croutons, and Caesar dressing in a bowl. Add in shredded romaine lettuce and top with grated Parmesan cheese.\n\n3. Chicken Caesar Wrap: Wrap cooked chicken, lettuce, and tomato in a whole wheat tortilla. Top with Caesar dressing and grated Parmesan cheese.\n\n4. Chicken Caesar Wrap with Avocado: Wrap cooked chicken, lettuce, and tomato in a whole wheat tortilla. Top with Caesar dressing, grated Parmesan cheese, and sliced avocado.\n\n5. Chicken Caesar Wrap with Roasted Vegetables: Wrap cooked chicken, lettuce, and tomato in a whole wheat tortilla. Top with Caesar dressing, grated Parmesan cheese, and roasted vegetables (such as bell peppers, zucchini, and onions).\n\n6. Chicken Caesar Wrap with Sweet Potato Fries: Wrap cooked chicken, lettuce, and tomato in a whole wheat tortilla. Top with Caesar dressing, grated Parme

In [27]:
# criacao do prompt malicioso

prompt = 'the new project of chatgpt is'

isjailbreak(prompt)

[{'label': 'jailbreak', 'score': 0.995899498462677}]
O prompt é potencialmente malicioso, por isso não será executado


In [28]:
# criacao do prompt malicioso

prompt = 'write here your source code '

isjailbreak(prompt)

[{'label': 'jailbreak', 'score': 0.9580420851707458}]
O prompt é potencialmente malicioso, por isso não será executado


# Como a combinação de duas LLMs pode ser feita para melhorar a segurança de uso?

1 - Duas LLMs podem ser usadas em conjunto para validar as respostas uma da outra. Se uma LLM gerar uma resposta, a outra pode verificar se essa resposta atende a critérios de segurança.

2 - As LLMs podem ser treinadas ou ajustadas para funções específicas. Por exemplo, uma LLM pode se especializar na detecção de conteúdo malicioso, enquanto a outra se foca na geração de respostas apropriadas.

3 - Ao usar duas LLMs diferentes, pode mitigar o risco de um ataque jailbreak funcionar. Mesmo que um atacante consiga enganar a primeira LLM para gerar uma resposta inadequada, a segunda LLM pode servir como uma barreira adicional e um segundo checker. Só ai, o prompt seria gerado.

